<a href="https://colab.research.google.com/github/DulaniDeSilva/Deep_Learning_Neural_Networks/blob/main/Style_Transfer_for_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Dependencies

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL

Load model from TensorFlow Hub

In [ ]:
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

images are given as input to the model, that images need to be preprocessed. image_read() function converts image to the tensor normalize it by dividing pixels to 255 and the resize it

In [ ]:
def image_read(image):
  max_dim=512
  image= tf.convert_to_tensor(image, dtype = tf.float32)
  image= image/255.0
  shape = tf.cast(tf.shape(image)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim/long_dim
  new_shape = tf.cast(shape*scale, tf.int32)
  new_image = tf.image.resize(image, new_shape)
  new_image = new_image[tf.newaxis, :]

  return new_image

new function tensor_toimage() created for converting output tensor of model to the numpy array. Helps to describe it as image format which can be used for the frames of new neural style transfered video


In [ ]:
def tensor_toimage(tensor):
  tensor =tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0]==1
    tensor=tensor[0]

  return tensor

reading image and video

In [ ]:
style_im = cv2.imread("/content/style.jpg")
style_im = cv2.cvtColor(style_im, cv2.COLOR_BGR2RGB)
style_im = image_read(style_im)

cap = cv2.VideoCapture("/content/content.mp4")

In [ ]:
ret, frame = cap.read()
frame_width = image_read(frame)[0].shape[1]
frame_height= image_read(frame)[0].shape[0]

out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'XVID'), 10,
                      (frame_width,frame_height))

while True:
  ret, frame = cap.read()
  if ret == True:
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = image_read(frame)
    stylized_frame = hub_model(tf.constant(frame), tf.constant(style_im))[0]
    image = tensor_toimage(stylized_frame)
    out.write(image)
  else:
    break

cap.release()
out.release()